In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('movie_metadata.csv')

In [3]:
df.drop_duplicates(inplace = True)

In [4]:
df.dropna(axis=0,subset=['director_name', 'num_critic_for_reviews','duration','director_facebook_likes','actor_2_name','actor_1_name','actor_3_name','facenumber_in_poster','num_user_for_reviews','language','country','plot_keywords'],inplace=True)

In [5]:
df["color"].fillna(df["color"].mode().iloc[0], inplace = True) 

In [6]:
df["content_rating"].fillna(df["content_rating"].mode().iloc[0], inplace = True) 

In [7]:
df["aspect_ratio"].fillna(df["aspect_ratio"].median(),inplace=True)

In [8]:
df["budget"].fillna(df["budget"].median(),inplace=True)

In [9]:
df["gross"].fillna(df["gross"].median(),inplace=True)

In [10]:
df['cast_crew_total_facebook_likes']=df['director_facebook_likes']+df['actor_3_facebook_likes']+df['actor_2_facebook_likes']+df['cast_total_facebook_likes']
df.head(2) 

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,cast_crew_total_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,6625.0
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,54913.0


In [11]:
df['critic_review_ratio']=df['num_critic_for_reviews']/df['num_user_for_reviews']

In [12]:
df.drop(columns=['genres','content_rating','director_facebook_likes','actor_3_facebook_likes','cast_total_facebook_likes','actor_2_facebook_likes','director_name','actor_1_name','actor_2_name','actor_3_name','movie_title','plot_keywords','movie_imdb_link','country','aspect_ratio','color','title_year','language','num_critic_for_reviews','num_user_for_reviews'],axis=1,inplace=True)

In [13]:
df.head()

,duration,actor_1_facebook_likes,gross,num_voted_users,facenumber_in_poster,budget,imdb_score,movie_facebook_likes,cast_crew_total_facebook_likes,critic_review_ratio
0,178.0,1000.0,760505847.0,886204,0.0,237000000.0,7.9,33000,6625.0,0.236739
1,169.0,40000.0,309404152.0,471220,0.0,300000000.0,7.1,0,54913.0,0.243942
2,148.0,11000.0,200074175.0,275868,1.0,245000000.0,6.8,85000,12254.0,0.605634
3,164.0,27000.0,448130642.0,1144337,0.0,250000000.0,8.5,164000,174759.0,0.301000
5,132.0,640.0,73058679.0,212204,1.0,263700000.0,6.6,24000,3510.0,0.626016


In [22]:
df.columns


Index(['duration', 'actor_1_facebook_likes', 'gross', 'num_voted_users',
       'facenumber_in_poster', 'budget', 'imdb_score', 'movie_facebook_likes',
       'cast_crew_total_facebook_likes', 'critic_review_ratio'],
      dtype='object')

In [15]:
#Label encoding the categorical columns
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#cat_list=['genres','content_rating']
#df[cat_list]=df[cat_list].apply(lambda x:le.fit_transform(x))

In [24]:
# split the dataset into dependent(X) and Independent(Y) datasets
X=df.drop(['imdb_score'],axis=1)
y = df['imdb_score']

In [25]:
# spliting the data into trainning and test dateset
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [26]:
# feature scaling using StandardScaler
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

X_train=ss.fit_transform(X_train)
X_test=ss.fit_transform(X_test)

In [27]:
import xgboost as xgb

model= xgb.XGBRegressor(n_estimators = 300)
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=300, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [28]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

predictions = model.predict(X_test)


In [29]:
#Exporting
import pickle

pickle.dump(model,open('model.pkl','wb'))